In [8]:
from heeps.config.read_config import read_config
from heeps.config.update_config import update_config
from heeps.pupil.create_stop import create_stop
from heeps.util.img_processing import crop_img, resize_img
from heeps.util.round2 import round2odd
from astropy.io import fits

In [9]:
# High Resolution Pupil: 20 micron per pixel; 45mm IMG ; 19.9mm LMS
conf = read_config()
npupil = {'L': update_config(**dict(read_config(), band='L'))['npupil'],
          'M': update_config(**dict(read_config(), band='M'))['npupil'],
         'HR': 1001,
     'HR_IMG': round2odd((45.0/38542*39998.8)/0.02),
     'HR_LMS': round2odd((19.9/38542*39998.8)/0.02),
     'HR_IMG_SPP': round2odd((45.0/38542*39998.8)/0.05)}
print(npupil)
# magnification factor for nhr (the super high res before dowscaling to npupil)
nhr_mag = 6 #8

{'L': 285, 'M': 227, 'HR': 1001, 'HR_IMG': 2335, 'HR_LMS': 1033, 'HR_IMG_SPP': 935}


# APP IMG-LM

In [ ]:
stop_params = (0.0209, 0.0209, 0.0291)
for band in ['HR_IMG']:#['L', 'M', 'HR']:
    conf['npupil'] = npupil[band]
    i = 10
    nhr = 2**i - 1
    while nhr < nhr_mag*conf['npupil']:
        i += 1
        nhr = 2**i - 1
        print(nhr, conf['npupil'])
    #####
    # crop 15 pix for CTE shrinkage
    nhr = 2**14 - 1
    conf['npupil'] = 10030
    mask = create_stop(conf['diam_ext'], conf['diam_int'], *stop_params, 
                        circ_ext=False, circ_int=False, nhr=nhr, **conf)
    print('shape =', mask.shape)
    mask = crop_img(mask, 10000)
    print('shape =', mask.shape)
    mask = resize_img(mask, npupil[band])
    print('shape =', mask.shape)
    #####
    fits.writeto('amp_APP_IMG_stop_%s_%s_dRext=%s_dRint=%s_dRspi=%s.fits'%
        (band, npupil[band], *stop_params), mask, overwrite=True)
    fits.writeto('int_APP_IMG_stop_%s_%s_dRext=%s_dRint=%s_dRspi=%s.fits'%
        (band, npupil[band], *stop_params), mask**2, overwrite=True)

# APP LMS

In [4]:
stop_params = (0.0241, 0.0241, 0.0308)
for band in ['HR_LMS']:#['L', 'M', 'HR']:
    conf['npupil'] = npupil[band]
    i = 10
    nhr = 2**i - 1
    while nhr < 2*nhr_mag*conf['npupil']:
        i += 1
        nhr = 2**i - 1
        print(nhr, conf['npupil'])
    mask = create_stop(conf['diam_ext'], conf['diam_int'], *stop_params, 
                        circ_ext=False, circ_int=False, nhr=nhr, **conf)
    #####
    # crop 15 pix for CTE shrinkage
    nhr = 2**14 - 1
    conf['npupil'] = 10030
    mask = create_stop(conf['diam_ext'], conf['diam_int'], *stop_params, 
                        circ_ext=False, circ_int=False, nhr=nhr, **conf)
    print('shape =', mask.shape)
    mask = crop_img(mask, 10000)
    print('shape =', mask.shape)
    mask = resize_img(mask, npupil[band])
    print('shape =', mask.shape)
    #####
    fits.writeto('amp_APP_LMS_stop_%s_%s_dRext=%s_dRint=%s_dRspi=%s.fits'%
        (band, npupil[band], *stop_params), mask, overwrite=True)
    fits.writeto('int_APP_LMS_stop_%s_%s_dRext=%s_dRint=%s_dRspi=%s.fits'%
        (band, npupil[band], *stop_params), mask**2, overwrite=True)

2047 1033
4095 1033
8191 1033
16383 1033
shape = (10030, 10030)
shape = (10000, 10000)
shape = (1033, 1033)
